In [60]:
import dotenv
dotenv.load_dotenv()

import sys
import os
import openai

openai.api_key = os.environ['OPENAI_KEY']


import json
import inspect

def safe_json_parse(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        # Handle the specific JSON parsing error
        print(json_str)
        return json_str
class GPTtoolCaller:
    def __init__(self, model_name="gpt-4-1106-preview", system_msg=None):
        self.model_name = model_name
        self.tools = {}
        self.system_msg = system_msg
        self.tool_definitions = []

        self.messages=[]
        if not self.system_msg is None:
            self.messages.append({'role':'system','content':self.system_msg})
    def register_tool(self, func):
        self.tools[func.__name__] = func
        params = inspect.signature(func).parameters
        param_info = {
            "type": "object",
            "properties": {},
            "required": list(params.keys())
        }
        for param in params.values():
            # Example of how to fill the parameter details
            # You might need to adjust this depending on your specific needs
            param_info["properties"][param.name] = {
                "type": "string",  # or other type based on your tool
                "description": param.name  # or a more detailed description
            }
        self.tool_definitions.append({
            "type": "function",
            "function": {
                "name": func.__name__,
                "description": func.__doc__,
                "parameters": param_info
                },
        })
        return func

    def query(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        # First GPT-4 call to potentially trigger a tool call
        while response is None or response['finish_reason'] != "stop":
            response = openai.ChatCompletion.create(
                model=self.model_name,
                messages=self.messages,            
                tools=self.tool_definitions,
                tool_choice="auto",
                
            )

            
            for choice in response['choices']:
                print(choice)
                message = choice.get('message')
                self.messages.append(message)
                tool_calls = message.get("tool_calls")
                if tool_calls:
                    for tool_call in tool_calls:
                        function_call = tool_call.get('function')
                        tool_name = function_call["name"]
                        arguments_json = function_call["arguments"]
                        print(tool_name,arguments_json)
                        # Use the safe JSON parsing method
                        parsed_arguments = safe_json_parse(arguments_json)

                        if "error" in parsed_arguments:
                            return parsed_arguments["error"]
                        elif tool_name in self.tools:
                            # Call the registered tool with the parsed arguments
                            tool_response = self.tools[tool_name](**parsed_arguments)

                            self.messages.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": tool_name,
                                "content": tool_response,
                            }) 


In [61]:

gpt_caller = GPTtoolCaller(system_msg="""
Please take the users input, do not output any dangerours commands.
please use multiple tool calls per request if possible
TAlk to yourself, between tool calls. this is your memory.
do not write messages to the user, use only the funtions you have been given 
if you cannot answer then please call the tool UNKNOWN_command
""")

import tempfile

@gpt_caller.register_tool
def UNKNOWN_command(command):
    """This tool is called when the system does not know how to handle a command"""
    print(f"UNKNOWN COMMAND: {command}")


@gpt_caller.register_tool
def write_file(path, content):
    """This tool writes content to a file at the given path"""
    with open(path, "w") as f:
        f.write(content)
    return f"Wrote content to {path}"

@gpt_caller.register_tool
def read_file(path):
    """This tool reads the content of a file at the given path"""
    with open(path, "r") as f:
        return f.read()

@gpt_caller.register_tool
def create_dir(path):
    """This tool creates a directory at the given path"""
    os.mkdir(path)
    return f"Created directory at {path}"

@gpt_caller.register_tool
def remove_dir(path):
    """This tool removes a directory at the given path"""
    os.rmdir(path)
    return f"Removed directory at {path}"

@gpt_caller.register_tool
def remove_file(path):
    """This tool removes a file at the given path"""
    os.remove(path)
    return f"Removed file at {path}"

@gpt_caller.register_tool
def move_file(path, new_path):
    """This tool moves a file from the given path to the new path"""
    os.rename(path, new_path)
    return f"Moved file from {path} to {new_path}"

@gpt_caller.register_tool
def copy_file(path, new_path):
    """This tool copies a file from the given path to the new path"""
    with open(path, "r") as f:
        content = f.read()
    with open(new_path, "w") as f:
        f.write(content)
    return f"Copied file from {path} to {new_path}"

@gpt_caller.register_tool
def append_file(path, content):
    """This tool appends content to a file at the given path"""
    with open(path, "a") as f:
        f.write(content)
    return f"Appended content to {path}"

@gpt_caller.register_tool
def replace_file(path, content):
    """This tool replaces the content of a file at the given path"""
    with open(path, "w") as f:
        f.write(content)
    return f"Replaced content of {path}"

@gpt_caller.register_tool
def create_file(path):
    """This tool creates a file at the given path"""
    with open(path, "w") as f:
        f.write("")
    return f"Created file at {path}"

@gpt_caller.register_tool
def get_temp_file():
    """This tool creates a temporary file and returns the path"""
    with tempfile.NamedTemporaryFile() as f:
        return f.name   

@gpt_caller.register_tool
def get_temp_dir():
    """This tool creates a temporary directory and returns the path"""
    return tempfile.mkdtemp()

@gpt_caller.register_tool
def backup_file(path):
    """This tool copies a backup of a file from the given path"""
    with open(path, "r") as f:
        content = f.read()
    new_path = path+'.backup'
    with open(new_path, "w") as f:
        f.write(content)
    return f"backed up file from {path} to {new_path}"


@gpt_caller.register_tool
def restore_file(path):
    """This tool restores a file from the backup at a given path"""
    with open(path, "r") as f:
        content = f.read()
    new_path = path.replace('.backup','')
    with open(new_path, "w") as f:
        f.write(content)
    return f"restored file from {path} to {new_path}"




@gpt_caller.register_tool   
def list_dir(path):
    """This tool lists the files and directories in the given path"""
    return "\n".join(os.listdir(path))

def run_chat():
    print("Welcome to the GPT-4 chatbot, type 'exit' to quit")
    while True:
        user_input = input("You: ")
        if user_input == "exit":
            break
        print("User:",user_input)
        response = gpt_caller.query(user_input)
        print("GPT-4: ",response)
run_chat()

Welcome to the GPT-4 chatbot, type 'exit' to quit
User: read main.py and make a version that is about intel, save it as inte_test.py
{
  "finish_reason": "tool_calls",
  "index": 0,
  "message": {
    "content": null,
    "role": "assistant",
    "tool_calls": [
      {
        "function": {
          "arguments": "{\"path\":\"main.py\"}",
          "name": "read_file"
        },
        "id": "call_MfiPhulMjEsWljVbmnIvLfAc",
        "type": "function"
      }
    ]
  }
}
read_file {"path":"main.py"}
{
  "finish_reason": "tool_calls",
  "index": 0,
  "message": {
    "content": "The file `main.py` has been successfully read. Now, I will modify the code to reflect that it is designed for Intel CPUs. After that, I will save the modified code as `intel_test.py`. Let's start modifying the code.",
    "role": "assistant",
    "tool_calls": [
      {
        "function": {
          "arguments": "{\"path\":\"intel_test.py\",\"content\":\"# intel_test.py\\nfrom gui import create_window, run_ev

KeyboardInterrupt: 